## Creating a Pandas DataFrame from the Downloaded Data

In [3]:
import os 
import pandas as pd
from bs4 import BeautifulSoup
from glob import glob

In [7]:
SCORES_DIR = 'data/scores'
box_scores = glob(f'{SCORES_DIR}/*.html')

In [8]:
len(box_scores)

2631

In [11]:
def parse_html(box_score):
  with open(box_score) as f:
    html = f.read()
  
  soup = BeautifulSoup(html)
  # remove tr.over_header and tr.over_header selected elements
  [s.decompose() for s in soup.select('tr.over_header')]
  [s.decompose() for s in soup.select('tr.thead')]
  return soup

In [13]:
def read_line_score(soup: BeautifulSoup):
  line_score = pd.read_html(str(soup), attrs={'id': 'line_score'})[0]
  cols = list(line_score.columns)
  cols[0] = 'team'
  cols[-1] = 'total'
  line_score.columns = cols

  line_score = line_score[['team', 'total']]
  return line_score

In [17]:
def read_stats(soup: BeautifulSoup, team: str, stat: str) -> pd.DataFrame:
  df = pd.read_html(str(soup), attrs={'id': f'box-{team}-game-{stat}'}, index_col=0)[0]
  df = df.apply(pd.to_numeric, errors='coerce')
  return df

In [47]:
def read_season_info(soup: BeautifulSoup):
  nav = soup.select('#bottom_nav_container')[0]
  hrefs = [a['href'] for a in nav.find_all('a')]
  season = os.path.basename(hrefs[1]).split('_')[0]
  return season

In [51]:
base_cols = None
games = []

for box_score in box_scores:
  soup = parse_html(box_score)
  line_score = read_line_score(soup)
  teams = list(line_score['team'])

  summaries = []
  for team in teams:
    basic = read_stats(soup, team, 'basic')
    advanced = read_stats(soup, team, 'advanced')

    totals = pd.concat([basic.iloc[-1, :], advanced.iloc[-1, :]])
    totals.index = totals.index.str.lower()

    maxes = pd.concat([basic.iloc[:-1, :].max(), advanced.iloc[:-1, :].max()])
    maxes.index = maxes.index.str.lower() + '_max'

    summary = pd.concat([totals, maxes])

    if base_cols is None:
      base_cols = list(summary.index.drop_duplicates(keep='first'))
      base_cols = [b for b in base_cols if 'bpm' not in b]
    
    summary = summary[base_cols]
    summaries.append(summary)
  summary = pd.concat(summaries, axis=1).T
  game = pd.concat([summary, line_score], axis=1)
  game['home'] = [0, 1]
  game_opp = game.iloc[::-1].reset_index()
  game_opp.columns += '_opp'

  full_game = pd.concat([game, game_opp], axis=1)
  full_game['season'] = read_season_info(soup)
  full_game['date'] = os.path.basename(box_score)[:8]
  full_game['date'] = pd.to_datetime(full_game['date'], format='%Y%m%d')

  full_game['won'] = full_game['total'] > full_game['total_opp']
  games.append(full_game)

  if len(games) % 100 == 0:
    print(f'{len(games)} / {len(box_scores)}')

In [52]:
full_game

,mp,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,265.0,265.0,36.0,87.0,0.414,11.0,34.0,0.324,31.0,41.0,...,66.7,32.3,187.0,122.0,UTA,117,1,2016,2016-02-23,False
1,265.0,265.0,38.0,74.0,0.514,10.0,24.0,0.417,31.0,38.0,...,30.0,39.9,155.0,126.0,HOU,114,0,2016,2016-02-23,True
